In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from plotly.offline import iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('../input/covid19-vaccine-adverse-reactions-vaers-dataset/Features.csv')
df1.head().style.set_properties(**{'background-color':'purple',
                                     'color': 'white'})

In [ ]:
df = pd.read_csv('../input/covid19-vaccine-adverse-reactions-vaers-dataset/Covid1-19_VAERS_Jan_June_2021.csv')
df.head(2).style.set_properties(**{'background-color':'purple',
                                     'color': 'white'})

In [ ]:
df.isnull().sum()

In [ ]:
##Code by Taha07  https://www.kaggle.com/taha07/data-scientists-jobs-analysis-visualization/notebook

from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'green',
                      height =2000,
                      width = 2000
                     ).generate(str(df["VAX_MANU"]))
plt.rcParams['figure.figsize'] = (12,12)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Vaccine Manufacturer")
plt.show()

In [ ]:
##Code by Taha07  https://www.kaggle.com/taha07/data-scientists-jobs-analysis-visualization/notebook

from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'yellow',
                      height =2000,
                      width = 2000
                     ).generate(str(df["VAX_NAME"]))
plt.rcParams['figure.figsize'] = (12,12)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Vaccine Name")
plt.show()

In [ ]:
##Code by Taha07  https://www.kaggle.com/taha07/data-scientists-jobs-analysis-visualization/notebook

from wordcloud import WordCloud
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(background_color = 'red',
                      height =2000,
                      width = 2000
                     ).generate(str(df["ALLERGIES"]))
plt.rcParams['figure.figsize'] = (12,12)
plt.axis("off")
plt.imshow(wordcloud)
plt.title("Allergies")
plt.show()

#Let's work with less features

In [ ]:
vaers = df[['VAERS_ID','SYMPTOMVERSION1','SYMPTOMVERSION2','SYMPTOMVERSION3','VAX_NAME','AGE_YRS', 'SEX', 'SYMPTOM_TEXT', 'DIED','ALLERGIES']]

In [ ]:
vaers.head()

In [ ]:
#Code by Lucas Abrahão https://www.kaggle.com/lucasabrahao/trabalho-manufatura-an-lise-de-dados-no-brasil

vaers["SEX"].value_counts().plot.bar(color=['blue', '#f5005a', '#7FFF00'], title='Participants Gender',);

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Histogram of Years of Age')
sns.histplot(data=vaers, x='AGE_YRS', bins=22, kde=True);

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Heatmap')
sns.heatmap(data=vaers.corr(), annot=True);

#Dealing with Missing Values

In [ ]:
# categorical features with missing values
categorical_nan = [feature for feature in vaers.columns if vaers[feature].isna().sum()>0 and vaers[feature].dtypes=='O']
print(categorical_nan)

In [ ]:
# replacing missing values in categorical features
for feature in categorical_nan:
    vaers[feature] = vaers[feature].fillna('None')

In [ ]:
vaers[categorical_nan].isna().sum()

In [ ]:
# Lets handle numerical features with nan value
numerical_nan = [feature for feature in vaers.columns if vaers[feature].isna().sum()>1 and vaers[feature].dtypes!='O']
numerical_nan

In [ ]:
vaers[numerical_nan].isna().sum()

In [ ]:
## Replacing the numerical Missing Values

for feature in numerical_nan:
    ## We will replace by using median since there are outliers
    median_value=vaers[feature].median()
    
    vaers[feature].fillna(median_value,inplace=True)
    
vaers[numerical_nan].isnull().sum()

In [ ]:
#Code by Bizen https://www.kaggle.com/hiro5299834/tps-apr-2021-deebtables/notebook

TARGET = 'ALLERGIES' #Target could Not be float otherwise will result in valueError: Unknown label type: 'continuous'. Even after the encoding.

label_cols = ['VAX_NAME', 'SEX', 'DIED']
numerical_cols = ['VAERS_ID', 'SYMPTOMVERSION1', 'SYMPTOMVERSION2', 'SYMPTOMVERSION3', 'AGE_YRS']

In [ ]:
#Code by Bizen https://www.kaggle.com/hiro5299834/tps-apr-2021-deebtables/notebook

from sklearn.preprocessing import LabelEncoder

def label_encoder(c):
    le = LabelEncoder()
    return le.fit_transform(c)

label_encoded_vaers = vaers[label_cols].apply(label_encoder)
numerical_vaers = vaers[numerical_cols]
target_vaers = vaers[TARGET]

vaers = pd.concat([label_encoded_vaers, numerical_vaers, target_vaers], axis=1)

In [ ]:
vaers.head()

#Code by Davegn  https://www.kaggle.com/davegn/breast-cancer-classification/notebook

In [ ]:
X = vaers.drop('ALLERGIES', axis=1)
y = vaers['ALLERGIES']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Model Imports

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

#Baseline model scores

In [ ]:
def fit_and_score(models, X_train, X_test, y_train, y_test):
    np.random.seed(42)
    
    model_scores = {}
    
    for name, model in models.items():
        model.fit(X_train,y_train)
        model_scores[name] = model.score(X_test,y_test)

    model_scores = pd.DataFrame(model_scores, index=['Score']).transpose()
    model_scores = model_scores.sort_values('Score')
        
    return model_scores

In [ ]:
models = {'LogisticRegression': LogisticRegression(max_iter=10000),
          'KNeighborsClassifier': KNeighborsClassifier(),
          'SVC': SVC(),
          'DecisionTreeClassifier': DecisionTreeClassifier(),
          'RandomForestClassifier': RandomForestClassifier(),
          'AdaBoostClassifier': AdaBoostClassifier(),
          'GradientBoostingClassifier': GradientBoostingClassifier(),
          'XGBClassifier': XGBClassifier(objective='binary:logistic',eval_metric=['logloss']),
          'XGBRFClassifier': XGBRFClassifier(objective='binary:logistic',eval_metric=['logloss']),
          'LGBMClassifier':LGBMClassifier()}

#After that line below my Notebook allocated too much memory. Alzheimer? Go to Plan B.

In [ ]:
#baseline_model_scores = fit_and_score(models, X_train, X_test, y_train, y_test)

In [ ]:
#baseline_model_scores

In [ ]:
#plt.figure(figsize=(20,10))
#sns.barplot(data=baseline_model_scores.sort_values('Score').T)
#plt.title('Baseline Model Precision Score')
#plt.xticks(rotation=90);

This are the follow models we can have a closer look at as it provides a better R2 score

LogisticRegression: ?????

KNeighborsClassifier ?????  

SVC ????    

#Plan B

In [ ]:
!pip install emot

In [ ]:
import emot

In [ ]:
text = 'Injection site swelling ☹️, redness, warm 💥 to the 😷'

ans = emot.emoji(text)

ans

In [ ]:
text1 = '😷Approximately 30 min 😲post vaccination ☹️administration patient demonstrated 😣 SOB and anxiousness.😫'

ans = emot.emoji(text1)

ans

In [ ]:
text2 = 'Felt✊ a little 👌 nauseous and a😬 little headache👀'

ans = emot.emoji(text2)

ans

In [ ]:
text3 = ' Sore 💪arm started ✊several hours 👌later approximately😲 9 PM 12/29/2020 2.☹️ Chills started 😔'

ans = emot.emoji(text3)

ans